In [60]:
# 원시 데이터 샘플링

import os
import shutil
from pathlib import Path

input_path_1 = './data_raw/train_label/TL_01.딸기_001.설향_01.정상'
input_path_2 = './data_raw/train_label/TL_01.딸기_001.설향_02.역병'
input_path_3 = './data_raw/train_label/TL_01.딸기_001.설향_03.시들음병'
input_path_4 = './data_raw/train_label/TL_01.딸기_001.설향_04.잎끝마름'
input_path_5 = './data_raw/train_label/TL_01.딸기_001.설향_05.황화'
output_path_1 = './data_raw/train_label/sampling_normal'
output_path_2 = './data_raw/train_label/sampling_blight'
output_path_3 = './data_raw/train_label/sampling_wilt'
output_path_4 = './data_raw/train_label/sampling_scorch'
output_path_5 = './data_raw/train_label/sampling_chlorosis'

input_paths = [input_path_1, input_path_2, input_path_3, input_path_4, input_path_5]
output_paths = [output_path_1, output_path_2, output_path_3, output_path_4, output_path_5]
num_files = 12000

def copy_files(input_path, output_path, num_files=12000):
    # output폴더를 여기서 생성하게 할 경우에는 아래의 Path 기능 사용
    # Path(output_path).mkdir(parents=True, exist_ok=True)
    
    files = [f for f in os.listdir(input_path) if f.endswith('.json')]
    files_to_copy = files[-num_files:]

    for file in files_to_copy:
        shutil.copy(os.path.join(input_path, file), os.path.join(output_path, file))

    return f"{len(files_to_copy)} files coppied from {input_path} to {output_path}"


In [61]:
copy_results = [copy_files(input_path, output_path, num_files) for input_path, output_path in zip(input_paths, output_paths)]
copy_results

KeyboardInterrupt: 

In [62]:
# 원시 데이터 데이터 프레임화(기존 작업 코드 재활용 후 여기서 현재 계획대로 데이터프레임 분리)
# 종류별로 작업 후 나중에 병합(concat, axis=0)

import json
import pandas as pd

def json_to_df(folder_path):
    df_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.json'):
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)

                df_list.append(pd.json_normalize(data))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {filename}: {e}")

    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
    else:
        print("No valid JSON files found")

    print('merged_df')
    return merged_df


In [63]:
folder_paths = [r'./data_raw/train_label/sampling_normal', r'./data_raw/train_label/sampling_blight',
                r'./data_raw/train_label/sampling_wilt', r'./data_raw/train_label/sampling_scorch',
                r'./data_raw/train_label/sampling_chlorosis']

merged_normal = json_to_df(folder_paths[0])
merged_blight = json_to_df(folder_paths[1])
merged_wilt = json_to_df(folder_paths[2])
merged_scorch = json_to_df(folder_paths[3])
merged_chlorosis = json_to_df(folder_paths[4])

merged_df
merged_df
merged_df
merged_df
merged_df


In [64]:
print(merged_normal.head(1))
print(merged_blight.head(1))
print(merged_wilt.head(1))
print(merged_scorch.head(1))
print(merged_chlorosis.head(1))

                                        environments  \
0  [{'farm_id': 'AI22_006', 'receive_date': '2022...   

                                         annotations  \
0  [{'coordinates': [256.48, 505.46, 401.62, 505....   

                                           licenses  \
0  [{'id': 1, 'name': 'CCMediaService', 'url': ''}]   

                                          categories  images.image_id  \
0  [{'id': 0, 'name': '잎', 'supercategory': 'none...           110703   

  images.farm_id images.crops_id images.crops images.kind_type  \
0       AI22_006    BERRY_55_008           딸기               설향   

             images.file_path  ... growth_index.fr2_cnt growth_index.fr3_cnt  \
0  /원천데이터/01.딸기/001.설향/01.정상/  ...                 None                 None   

  growth_index.fr_weight etc_infor.crops_id etc_infor.create_date  \
0                   None       BERRY_55_008                         

  etc_infor.inform images.stem images.grpoint images.fruit images.flower  
0       

In [68]:
merged_normal.to_csv('./data_raw/train_label/merged_df/merged_raw_normal.csv', encoding='utf-8', index=False)
merged_blight.to_csv('./data_raw/train_label/merged_df/merged_raw_blight.csv', encoding='utf-8', index=False)
merged_wilt.to_csv('./data_raw/train_label/merged_df/merged_raw_wilt.csv', encoding='utf-8', index=False)
merged_scorch.to_csv('./data_raw/train_label/merged_df/merged_raw_scorch.csv', encoding='utf-8', index=False)
merged_chlorosis.to_csv('./data_raw/train_label/merged_df/merged_raw_chlorosis.csv', encoding='utf-8', index=False)

In [1]:
# 데이터프레임 정제(df_main, df_bbox)
"""
raw 데이터프레임 컬럼 추출 요소
*environments: value_{ti, hi, ci, ir, sr, tl, ei, pl, cl, el, hl, pi, RP}
*annotations
*categories
*width
*height
*images.fname
*images.disease_class
*images.disease_cause_method
"""

import pandas as pd
import json
import numpy as np

raw_normal = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_normal.csv', encoding='utf-8')
raw_blight = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_blight.csv', encoding='utf-8')
raw_wilt = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_wilt.csv', encoding='utf-8')
raw_scorch = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_scorch.csv', encoding='utf-8')
raw_chlorosis = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_chlorosis.csv', encoding='utf-8')


In [2]:
raw_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   environments                 12000 non-null  object 
 1   annotations                  12000 non-null  object 
 2   licenses                     12000 non-null  object 
 3   categories                   12000 non-null  object 
 4   images.image_id              12000 non-null  int64  
 5   images.farm_id               12000 non-null  object 
 6   images.crops_id              12000 non-null  object 
 7   images.crops                 12000 non-null  object 
 8   images.kind_type             12000 non-null  object 
 9   images.file_path             12000 non-null  object 
 10  images.fname                 12000 non-null  object 
 11  images.fext                  12000 non-null  object 
 12  images.width                 12000 non-null  int64  
 13  images.height   

In [24]:
def extract_cols(df, col_list):
    df_sel = df[col_list].copy()
    df_sel['annotations'] = df_sel['annotations'].apply(lambda x: json.loads(x.replace("'", "\"")))
    df_sel['categories'] = df_sel['categories'].apply(lambda x: json.loads(x.replace("'", "\"")))
    df_sel['environments'] = df_sel['environments'].apply(lambda x: json.dumps(x))
    df_sel['environments'] = df_sel['environments'].apply(json.loads)

    max_len = df_sel.loc[df_sel['annotations'].str.len().idxmax()]
    max_len_val = max_len['annotations']
    data = max_len_val

    if data and isinstance(data, list):
        item = data[-1]
        cat_id = item['category_id']
        print(cat_id)
    else:
        print("error")

    length = cat_id + 1
    zero_box = [0,0,0,0]
    unknown_status = 'F'

    for i in range(length):
        df_sel[f'bbox_{i}'] = None
        df_sel[f'status_{i}'] = None
        df_sel[f'name_{i}'] = None

        df_sel[f'ti_value_{i}'] = None
        df_sel[f'hi_value_{i}'] = None
        df_sel[f'ci_value_{i}'] = None
        df_sel[f'ir_value_{i}'] = None
        df_sel[f'tl_value_{i}'] = None
        df_sel[f'ei_value_{i}'] = None
        df_sel[f'pl_value_{i}'] = None
        df_sel[f'sr_value_{i}'] = None
        df_sel[f'cl_value_{i}'] = None
        df_sel[f'el_value_{i}'] = None
        df_sel[f'hl_value_{i}'] = None
        df_sel[f'pi_value_{i}'] = None
        df_sel[f'rp_value_{i}'] = None


    for idx, row in df_sel.iterrows():
        annotation_data = row['annotations']
        for k in range(len(annotation_data)):
            if k < len(annotation_data):
                df_sel.at[idx, f'bbox_{k}'] = annotation_data[k]['bbox']
                df_sel.at[idx, f'status_{k}'] = annotation_data[k]['disease_status']
            else:
                df_sel.at[idx, f'bbox_{k}'] = zero_box
                df_sel.at[idx, f'status_{k}'] = unknown_status

    for idx, row in df_sel.iterrows():
        name_data = row['categories']
        for k in range(len(name_data)):
            if k < len(name_data):
                df_sel.at[idx, f'name_{k}'] = name_data[k]['name']
            else:
                df_sel.at[idx, f'name_{k}'] = 'unknown'

    for idx, row in df_sel.iterrows():
        value_data = row['environments']
        if isinstance(value_data, str):
            value_data = json.loads(value_data)
        for k in range(len(value_data)):
            if k < len(value_data):
                df_sel.at[idx, f'ti_value_{k}'] = value_data[k]['ti_value']
                df_sel.at[idx, f'hi_value_{k}'] = value_data[k]['hi_value']
                df_sel.at[idx, f'ci_value_{k}'] = value_data[k]['ci_value']
                df_sel.at[idx, f'ir_value_{k}'] = value_data[k]['ir_value']
                df_sel.at[idx, f'tl_value_{k}'] = value_data[k]['tl_value']
                df_sel.at[idx, f'ei_value_{k}'] = value_data[k]['ei_value']
                df_sel.at[idx, f'pl_value_{k}'] = value_data[k]['pl_value']
                df_sel.at[idx, f'sr_value_{k}'] = value_data[k]['sr_value']
                df_sel.at[idx, f'cl_value_{k}'] = value_data[k]['cl_value']
                df_sel.at[idx, f'el_value_{k}'] = value_data[k]['el_value']
                df_sel.at[idx, f'hl_value_{k}'] = value_data[k]['hl_value']
                df_sel.at[idx, f'pi_value_{k}'] = value_data[k]['pi_value']
                df_sel.at[idx, f'rp_value_{k}'] = value_data[k]['rp_value']
            else:
                df_sel.at[idx, f'ti_value_{k}'] = 'no_value'
                df_sel.at[idx, f'hi_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ci_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ir_value_{k}'] = 'no_value'
                df_sel.at[idx, f'tl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ei_value_{k}'] = 'no_value'
                df_sel.at[idx, f'pl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'sr_value_{k}'] = 'no_value'
                df_sel.at[idx, f'cl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'el_value_{k}'] = 'no_value'
                df_sel.at[idx, f'hl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'pi_value_{k}'] = 'no_value'
                df_sel.at[idx, f'rp_value_{k}'] = 'no_value'


    df_sel.drop(columns=['environments','annotations', 'categories'], axis=1, inplace=True)

    return df_sel



In [2]:
normal_sample = raw_normal[['environments', 'annotations']][:5].copy()
normal_sample.head(1)

,environments,annotations
0,"[{'farm_id': 'AI22_006', 'receive_date': '2022...","[{'coordinates': [256.48, 505.46, 401.62, 505...."


In [3]:
json_str = normal_sample['environments'][0]
str_corrected = json_str.replace("None", "null").replace("'", "\"")

env = json.loads(str_corrected)

print(type(env))
print(env)

<class 'list'>
[{'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:00:26', 'id': 0, 'ti_value': '16.7', 'hi_value': '93.4', 'ci_value': '595', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:01:23', 'id': 1, 'ti_value': '16.7', 'hi_value': '93.5', 'ci_value': '597', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:02:24', 'id': 2, 'ti_value': '16.7', 'hi_value': '93.5', 'ci_value': '597', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006

In [25]:
print(type(env[0]['ti_value']))

<class 'str'>


In [ ]:
"""
각 _value에 대한 빈 리스트 생성(각 _value를 key, _value에 할당된 값들의 리스트를 value로 하는 딕셔너리 생성하기)
    # _value명 리스트를 추출하고, 각 요소를 변수로 하는 빈 리스트 생성은 동적 변수 할당이라 권장되지 않음
각 딕셔너리에서 해당 값을 대상 빈 리스트에 append -> 각 값들이 집합한 리스트 생성
리스트의 요소 타입을 str -> float으로 변환(현재 딕셔너리에 저장된 각 숫자값들은 str로 되어있음)
리스트별로 요소의 값들의 평균을 계산(sum/len)
각 _value이름을 갖는 컬럼을 생성하고 거기에 값 입력

위의 과정을 데이터프레임 전체에 적용하는 함수 생성
"""

145


In [4]:
elements_value = env[0]
name_value = []
for key, value in elements_value.items():
    if key.endswith('_value'):
        name_value.append(key)

print(name_value)

['ti_value', 'hi_value', 'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value']


In [5]:
value_list = {}

for dictionary in env:
    for key, value in dictionary.items():
        if key in name_value:
            if key not in value_list:
                value_list[key] = [value]
            else:
                value_list[key].append(value)

value_list


{'ti_value': ['16.7', '16.7', '16.7', '16.7', '16.7'],
 'hi_value': ['93.4', '93.5', '93.5', '93.5', '93.5'],
 'ci_value': ['595', '597', '597', '597', '597'],
 'ir_value': [None, None, None, None, None],
 'tl_value': [None, None, None, None, None],
 'ei_value': ['0.07', '0.07', '0.07', '0.07', '0.07'],
 'pl_value': ['6.2', '6.2', '6.2', '6.2', '6.2'],
 'sr_value': ['0', '0', '0', '0', '0'],
 'cl_value': ['20.2', '20.2', '20.2', '20.2', '20.2'],
 'el_value': ['3.20', '3.20', '3.20', '3.20', '3.20'],
 'hl_value': ['3.9', '3.9', '3.9', '3.9', '3.9'],
 'pi_value': ['11.0', '11.0', '11.0', '11.0', '11.0'],
 'rp_value': [None, None, None, None, None]}

In [6]:
def extract_values(dict_list, value_names):
    value_dict = {}
    for dictionary in dict_list:
        for key, value in dictionary.items():
            if key in value_names:
                if key not in value_dict:
                    value_dict[key] = [value]
                else:
                    value_dict[key].append(value)
    return value_dict

row_1 = extract_values(env, name_value)
print(row_1)

{'ti_value': ['16.7', '16.7', '16.7', '16.7', '16.7'], 'hi_value': ['93.4', '93.5', '93.5', '93.5', '93.5'], 'ci_value': ['595', '597', '597', '597', '597'], 'ir_value': [None, None, None, None, None], 'tl_value': [None, None, None, None, None], 'ei_value': ['0.07', '0.07', '0.07', '0.07', '0.07'], 'pl_value': ['6.2', '6.2', '6.2', '6.2', '6.2'], 'sr_value': ['0', '0', '0', '0', '0'], 'cl_value': ['20.2', '20.2', '20.2', '20.2', '20.2'], 'el_value': ['3.20', '3.20', '3.20', '3.20', '3.20'], 'hl_value': ['3.9', '3.9', '3.9', '3.9', '3.9'], 'pi_value': ['11.0', '11.0', '11.0', '11.0', '11.0'], 'rp_value': [None, None, None, None, None]}


In [7]:
normal_sample['values'] = None

for idx, row in normal_sample.iterrows():
    json_str = row['environments']
    str_corrected = json_str.replace("None", "null").replace("'", "\"")
    env_values = json.loads(str_corrected)
    env_value_dict = extract_values(env_values, name_value)
    normal_sample.at[idx, 'values'] = env_value_dict

normal_sample.head(1)


,environments,annotations,values
0,"[{'farm_id': 'AI22_006', 'receive_date': '2022...","[{'coordinates': [256.48, 505.46, 401.62, 505....","{'ti_value': ['16.7', '16.7', '16.7', '16.7', ..."


In [44]:
for idx, row in normal_sample.iterrows():
    len_values = row['values']
    print(len(len_values))
    print(len_values)

print(normal_sample.info())
print(normal_sample['values'][0])

13
{'ti_value': ['16.7', '16.7', '16.7', '16.7', '16.7'], 'hi_value': ['93.4', '93.5', '93.5', '93.5', '93.5'], 'ci_value': ['595', '597', '597', '597', '597'], 'ir_value': [None, None, None, None, None], 'tl_value': [None, None, None, None, None], 'ei_value': ['0.07', '0.07', '0.07', '0.07', '0.07'], 'pl_value': ['6.2', '6.2', '6.2', '6.2', '6.2'], 'sr_value': ['0', '0', '0', '0', '0'], 'cl_value': ['20.2', '20.2', '20.2', '20.2', '20.2'], 'el_value': ['3.20', '3.20', '3.20', '3.20', '3.20'], 'hl_value': ['3.9', '3.9', '3.9', '3.9', '3.9'], 'pi_value': ['11.0', '11.0', '11.0', '11.0', '11.0'], 'rp_value': [None, None, None, None, None]}
13
{'ti_value': ['16.7', '16.7', '16.7', '16.7', '16.7'], 'hi_value': ['93.4', '93.5', '93.5', '93.5', '93.5'], 'ci_value': ['595', '597', '597', '597', '597'], 'ir_value': [None, None, None, None, None], 'tl_value': [None, None, None, None, None], 'ei_value': ['0.07', '0.07', '0.07', '0.07', '0.07'], 'pl_value': ['6.2', '6.2', '6.2', '6.2', '6.2'], 's

In [8]:
for index, row in normal_sample.iterrows():
    values = row['values']
    for key, value in values.items():
        print(key, value)

ti_value ['16.7', '16.7', '16.7', '16.7', '16.7']
hi_value ['93.4', '93.5', '93.5', '93.5', '93.5']
ci_value ['595', '597', '597', '597', '597']
ir_value [None, None, None, None, None]
tl_value [None, None, None, None, None]
ei_value ['0.07', '0.07', '0.07', '0.07', '0.07']
pl_value ['6.2', '6.2', '6.2', '6.2', '6.2']
sr_value ['0', '0', '0', '0', '0']
cl_value ['20.2', '20.2', '20.2', '20.2', '20.2']
el_value ['3.20', '3.20', '3.20', '3.20', '3.20']
hl_value ['3.9', '3.9', '3.9', '3.9', '3.9']
pi_value ['11.0', '11.0', '11.0', '11.0', '11.0']
rp_value [None, None, None, None, None]
ti_value ['16.7', '16.7', '16.7', '16.7', '16.7']
hi_value ['93.4', '93.5', '93.5', '93.5', '93.5']
ci_value ['595', '597', '597', '597', '597']
ir_value [None, None, None, None, None]
tl_value [None, None, None, None, None]
ei_value ['0.07', '0.07', '0.07', '0.07', '0.07']
pl_value ['6.2', '6.2', '6.2', '6.2', '6.2']
sr_value ['0', '0', '0', '0', '0']
cl_value ['20.2', '20.2', '20.2', '20.2', '20.2']
el_va

In [9]:
for index, row in normal_sample.iterrows():
    values = row['values']
    for key, value in values.items():
        if key not in normal_sample.columns:
            normal_sample[key] = pd.NA
        normal_sample.at[index, key] = value

normal_sample.head(1)

,environments,annotations,values,ti_value,hi_value,ci_value,ir_value,tl_value,ei_value,pl_value,sr_value,cl_value,el_value,hl_value,pi_value,rp_value
0,"[{'farm_id': 'AI22_006', 'receive_date': '2022...","[{'coordinates': [256.48, 505.46, 401.62, 505....","{'ti_value': ['16.7', '16.7', '16.7', '16.7', ...","[16.7, 16.7, 16.7, 16.7, 16.7]","[93.4, 93.5, 93.5, 93.5, 93.5]","[595, 597, 597, 597, 597]","[None, None, None, None, None]","[None, None, None, None, None]","[0.07, 0.07, 0.07, 0.07, 0.07]","[6.2, 6.2, 6.2, 6.2, 6.2]","[0, 0, 0, 0, 0]","[20.2, 20.2, 20.2, 20.2, 20.2]","[3.20, 3.20, 3.20, 3.20, 3.20]","[3.9, 3.9, 3.9, 3.9, 3.9]","[11.0, 11.0, 11.0, 11.0, 11.0]","[None, None, None, None, None]"


In [18]:
## 지금까지의 value 분리 작업을 함수 조합으로 합치기

def json_data_to_pandas(df, target_col, saving_col):
    df[saving_col] = None

    for idx, row in df.iterrows():
        json_str = row[target_col]
        str_corrected = json_str.replace("None", "null").replace("'", "\"")
        corrected_data = json.loads(str_corrected)
        df.at[idx, saving_col] = corrected_data
    
    dict_format = df[saving_col][0]
    extraction_keys = dict_format[0].keys()
    key_list = list(extraction_keys)
    df_pandas_type = df.drop([target_col], axis=1)

    print(type(df[saving_col]))
    return df_pandas_type, key_list


def extraction_env_values(df_pandas_type, key_list):
    df = df_pandas_type
    df['value_types'] = None
    value_names = []
    
    for i in range(len(key_list)):
        candidate = key_list[i]
        if candidate.endswith('_value'):
            value_names.append(candidate)
    
    for idx, row in df.iterrows():
        value_dict = {}
        dict_list = row['values']
        for dictionary in dict_list:
            for key, value in dictionary.items():
                if key in value_names:
                    if key not in value_dict:
                        value_dict[key] = [value]
                    else:
                        value_dict[key].append(value)
        df.at[idx, 'value_types'] = value_dict
    
    for idx, row in df.iterrows():
        value_to_split = row['value_types']
        for key, value in value_to_split.items():
            if key not in df.columns:
                df[key] = pd.NA
            df.at[idx, key] = value

    # df_splited_env_values = df.drop(['values'], axis=1)
    df_splited_env_values = df
    print(df_splited_env_values.columns)
    return df_splited_env_values

# def calculate_env_values(df):
#     for col in df.columns:
#         if col.endswith('_value'):
#             df[col] = df[col].apply(lambda x: np.mean([float(i) for i in x if i is not None]) if any(i is not None for i in x) else None)

    
#     return df

In [3]:
sample_check = raw_normal[['environments', 'annotations']][:5].copy()

sample_df, env_value_list = json_data_to_pandas(sample_check, target_col='environments', saving_col='values')
sample_df_anno, anno_value_list = json_data_to_pandas(sample_check, target_col='annotations', saving_col='anno_values')

print(sample_df['values'].head(1))
print(env_value_list)
print(sample_df_anno['anno_values'].head(1))
print(anno_value_list)
print(sample_df.columns)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
0    [{'farm_id': 'AI22_006', 'receive_date': '2022...
Name: values, dtype: object
['farm_id', 'receive_date', 'id', 'ti_value', 'hi_value', 'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value']
0    [{'coordinates': [256.48, 505.46, 401.62, 505....
Name: anno_values, dtype: object
['coordinates', 'area', 'bbox', 'isCrowd', 'id', 'image_id', 'category_id', 'disease_status']
Index(['annotations', 'values'], dtype='object')


In [19]:
sample_check = raw_normal[['environments', 'annotations']][:400].copy()

sample_df, env_value_list = json_data_to_pandas(sample_check, target_col='environments', saving_col='values')

df_result = extraction_env_values(sample_df, env_value_list)
print(df_result['value_types'][0])
df_result.to_csv('./normal_env_sample_check.csv', encoding='utf-8', index=False)

<class 'pandas.core.series.Series'>
Index(['annotations', 'values', 'value_types', 'ti_value', 'hi_value',
       'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value',
       'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
{'ti_value': ['16.7', '16.7', '16.7', '16.7', '16.7'], 'hi_value': ['93.4', '93.5', '93.5', '93.5', '93.5'], 'ci_value': ['595', '597', '597', '597', '597'], 'ir_value': [None, None, None, None, None], 'tl_value': [None, None, None, None, None], 'ei_value': ['0.07', '0.07', '0.07', '0.07', '0.07'], 'pl_value': ['6.2', '6.2', '6.2', '6.2', '6.2'], 'sr_value': ['0', '0', '0', '0', '0'], 'cl_value': ['20.2', '20.2', '20.2', '20.2', '20.2'], 'el_value': ['3.20', '3.20', '3.20', '3.20', '3.20'], 'hl_value': ['3.9', '3.9', '3.9', '3.9', '3.9'], 'pi_value': ['11.0', '11.0', '11.0', '11.0', '11.0'], 'rp_value': [None, None, None, None, None]}


In [7]:
value_type_list = []
for value in df_result.columns:
    value_type = value
    if value_type.endswith('_value'):
        value_type_list.append(value_type)

print(value_type_list)

['ti_value', 'hi_value', 'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value']


In [10]:
val_cols = value_type_list
avg_test = df_result.copy()
df_values = df_result[[val_cols]]

def calculate_avg(df, col):
    for idx, row in df.iterrows():
        scores = row[col]
        val_box = []
        for val in scores:
            if val is not None:
                num_type = float(val)
                val_box.append(num_type)
        score_avg = np.mean(val_box)
        df.at[idx, col] = score_avg
    return df



for v in val_cols:
    




AttributeError: 'Series' object has no attribute 'iterrows'

In [1]:
import pandas as pd
import numpy as np
import ast

check = pd.read_csv('./normal_env_sample_check.csv', encoding='utf-8')
check.head(1)

,annotations,values,value_types,ti_value,hi_value,ci_value,ir_value,tl_value,ei_value,pl_value,sr_value,cl_value,el_value,hl_value,pi_value,rp_value
0,"[{'coordinates': [256.48, 505.46, 401.62, 505....","[{'farm_id': 'AI22_006', 'receive_date': '2022...","{'ti_value': ['16.7', '16.7', '16.7', '16.7', ...","['16.7', '16.7', '16.7', '16.7', '16.7']","['93.4', '93.5', '93.5', '93.5', '93.5']","['595', '597', '597', '597', '597']","[None, None, None, None, None]","[None, None, None, None, None]","['0.07', '0.07', '0.07', '0.07', '0.07']","['6.2', '6.2', '6.2', '6.2', '6.2']","['0', '0', '0', '0', '0']","['20.2', '20.2', '20.2', '20.2', '20.2']","['3.20', '3.20', '3.20', '3.20', '3.20']","['3.9', '3.9', '3.9', '3.9', '3.9']","['11.0', '11.0', '11.0', '11.0', '11.0']","[None, None, None, None, None]"


In [36]:
import ast
a = check['ti_value']
print(type(a))
print(type(a[0]))

b = a[0]
b_list = ast.literal_eval(b)
print(type(b_list))
print(b_list)
print(type(b_list[0]))
for i in b_list:


<class 'pandas.core.series.Series'>
<class 'str'>
<class 'list'>
['16.7', '16.7', '16.7', '16.7', '16.7']
<class 'str'>


In [2]:
## 정리
"""
1. json -> df(pandas에서 편집 가능한 딕셔너리 리스트 형태로 변환) -> 딕셔너리 리스트의 각 딕셔너리 분리 -> 분리한 각 딕셔너리에서 키와 값 분리(값은 리스트 형태)
2. 위의 과정에서 생성되는 리스트 형태의 값들은 '리스트' 타입이 아니라 '리스트의 형태를 갖는 문자열' 타입이다.
3. '리스트 형태를 갖는 문자열' 데이터들을 리스트로 변환(ast.literal_eval사용); None에 대한 예외처리 필요
4. 리스트로 변환한 각 데이터의 구성 요소가 숫자형태의 문자일 경우 -> 숫자의 형태에 맞게 정수 또는 실수로 변환
5. 위의 과정을 모두 거쳐야 비로소 대상 컬럼의 각 셀 데이터가 숫자를 포함한 리스트 형태가 된다.
    *이 때, astype는 사용할 수 없다. 해당 함수는 데이터프레임의 각 셀이 'a', '1'과 같이 단일 형태의 데이터일때의 일괄 형변환에 사용할 수 있고, 리스트와 같이 복합구성에 대해서는 안됨
        *대상 셀의 데이터가 숫자형 문자 리스트라면, 해당 셀 데이터는 단일 리스트 타입의 데이터로 인식하기 때문.
"""

## 이 블록에서 해야 하는 작업
    # '_value' 컬럼값들의 타입 변경(str -> list)
    # 리스트로 변환된 각 데이터 구성 요소들을 float로 변경
    # float로 변환된 각 리스트의 평균 계산(np.mean())
    # 위의 과정에서 None 처리에 대한 예외조건 필요

def str_to_num(string_list):
    list_obj = ast.literal_eval(string_list)
    if all(item is None for item in list_obj):
        return None
    else:
        list_num = [float(item) for item in list_obj if item is not None]

        return list_num
   
def string_to_number(df):
    col_list = [col for col in df.columns if col.endswith('_value')]
    
    for col in col_list:
        df[col] = df[col].apply(str_to_num)
        
    return df                

In [3]:
check_float = string_to_number(check)
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   annotations  400 non-null    object
 1   values       400 non-null    object
 2   value_types  400 non-null    object
 3   ti_value     400 non-null    object
 4   hi_value     400 non-null    object
 5   ci_value     400 non-null    object
 6   ir_value     0 non-null      object
 7   tl_value     81 non-null     object
 8   ei_value     400 non-null    object
 9   pl_value     400 non-null    object
 10  sr_value     400 non-null    object
 11  cl_value     400 non-null    object
 12  el_value     400 non-null    object
 13  hl_value     400 non-null    object
 14  pi_value     400 non-null    object
 15  rp_value     0 non-null      object
dtypes: object(16)
memory usage: 50.1+ KB


In [6]:
check_list = [col for col in check_float if col.endswith('_value')]

type_list_origin = []
type_list_float = []
for types in check_list:
    for col in check.columns:
        if col == types:
            type_class = type(check[col][0])
            type_list_origin.append(type_class)
    for col_2 in check_float.columns:
        if col_2 == types:
            type_class_2 = type(check_float[col_2][0])
            type_list_float.append(type_class_2)

print(type_list_origin)
print(type_list_float)


[<class 'list'>, <class 'list'>, <class 'list'>, <class 'NoneType'>, <class 'NoneType'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'NoneType'>]
[<class 'list'>, <class 'list'>, <class 'list'>, <class 'NoneType'>, <class 'NoneType'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'NoneType'>]


In [47]:
value_types = []
for col in check.columns:
    if col.endswith('_value'):
        value_types.append(col)
print(type(check['ti_value']))
print(type(check[value_types[0]]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [42]:
print(type(check['ti_value'][0]))
print(type(check['ti_value_float'][0]))
print(check['ti_value'][0])
print(check['ti_value_float'][0])
print(np.mean(check['ti_value_float'][0]))
print(check.columns)

float_list = []
for float in check.columns:
    if float.endswith('_float'):
        float_list.append(float)

for types in float_list:
    print(type(check[types][0]))


<class 'str'>
<class 'list'>
['16.7', '16.7', '16.7', '16.7', '16.7']
[16.7, 16.7, 16.7, 16.7, 16.7]
16.7
Index(['annotations', 'values', 'value_types', 'ti_value', 'hi_value',
       'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value',
       'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value', 'ti_list',
       'ti_value_float', 'ti_value_float_float'],
      dtype='object')
<class 'list'>
<class 'NoneType'>


In [6]:
check_list = []
for i in range(len(env_value_list)):
    value = env_value_list[i]
    if value.endswith('value'):
        check_list.append(value)

print(check_list)


['ti_value', 'hi_value', 'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value']


In [ ]:
"""
bbox, name, status 분리 작업

딕셔너리 집합을 풀어서 각각 row로 할당(공유 정보는 동일하게 컬럼에 데이터 할당하기: 예; images.fname)
각 row에서 bbox, name, status 분리하여 각 이름을 갖는 컬럼에 할당
id할당
5개 각각에 위의 작업 실행 후 최종적으로 main_df, info_df 분리하여 통합 데이터프레임 2개로 정리하기
"""




['ti_value', 'hi_value', 'ci_value', 'ir_value', 'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value', 'hl_value', 'pi_value', 'rp_value']


In [ ]:
# 라벨 데이터에 해당하는 원본 이미지 파일 복사

In [ ]:
# df_main, df_bbox 결과 확인